In [39]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# ==========================================================
# GLOBAL PARAMETERS
# ==========================================================
N = 20
SINK = 0
Pmax = 0.316
noise_power = 1e-9

packet_size = 1024
bandwidth = 5e6
slot_time = 0.01
SNR_dB = np.arange(0, 31, 5)
ROUNDS = 400
packet_arrival_prob = 0.4
load = 0.6

In [41]:
# RL
Q = np.zeros((N, 3))
alpha_rl, gamma_rl = 0.1, 0.9


In [42]:
# ==========================================================
# NODE INITIALIZATION (AD-HOC NETWORK)
# ==========================================================
def init_nodes():
    nodes = []
    for i in range(N):
        nodes.append({
            "id": i,
            "energy": np.random.uniform(60, 100),
            "power": np.random.uniform(0.05, 0.12),
            "h2": 0.0,
            "traffic": 0,
            "parent": None,
            "role": "leaf"
        })
    return nodes


In [43]:

# ==========================================================
# CHANNEL MODEL
# ==========================================================
def channel_model(nodes):
    d = np.random.uniform(10, 100, N)
    fading = (np.random.randn(N) + 1j*np.random.randn(N)) / np.sqrt(2)
    h = np.sqrt(d**-3) * fading
    for i in range(N):
        nodes[i]["h2"] = np.abs(h[i])**2

In [44]:
# ==========================================================
# HMBO (POPULATION + MIGRATION)
# ==========================================================
def hmbo(nodes, pop_size=8, iters=12):
    population = []
    for _ in range(pop_size):
        population.append([
            np.clip(n["power"] * np.random.uniform(0.7,1.3),0.01,Pmax)
            for n in nodes
        ])

    for _ in range(iters):
        fitness = []
        for p in population:
            f = sum(nodes[i]["h2"]*p[i] - 0.02*p[i] for i in range(N))
            fitness.append(f)

        best = population[np.argmax(fitness)]

        for k in range(pop_size):
            for i in range(N):
                population[k][i] = np.clip(
                    0.7*best[i] + 0.3*population[k][i] +
                    np.random.uniform(-0.01,0.01),
                    0.01, Pmax
                )

    for i in range(N):
        nodes[i]["power"] = best[i]

In [45]:
# ==========================================================
# TTPA – TREE TOPOLOGY POWER ALLOCATION
# ==========================================================
def build_tree(nodes):
    nodes[SINK]["parent"] = None
    nodes[SINK]["role"] = "sink"
    for i in range(1, N):
        parent = np.argmax([nodes[j]["energy"] for j in range(i)])
        nodes[i]["parent"] = parent
        nodes[parent]["role"] = "relay"

In [46]:
# ==========================================================
# RELAY SELECTION
# ==========================================================
def relay_selection(nodes):
    for n in nodes:
        if n["id"] == SINK:
            continue
        score = 0.5*n["energy"] + 0.3*n["h2"] - 0.2*n["traffic"]
        n["role"] = "relay" if score > 50 else "leaf"


In [47]:
# ==========================================================
# NOMA CLUSTERING
# ==========================================================
def noma_clusters(active, nodes):
    idx = sorted(active, key=lambda i: nodes[i]["h2"])
    return [(idx[i], idx[-i-1]) for i in range(len(idx)//2)]

In [48]:
# ==========================================================
# SIC RECEIVER
# ==========================================================
def sic(nodes, clusters, snr_db):
    noise = noise_power / (10**(snr_db/10))
    sinr_th = 0.04
    success = {}
    sinr_values = []

    for c in clusters:
        rx = [(nodes[i]["power"]*nodes[i]["h2"], i) for i in c]
        rx.sort(reverse=True)
        interf = sum(p for p,_ in rx)
        load_factor = 1 + load*np.random.rand()

        for p,i in rx:
            sinr = p / (load_factor*(interf - p) + noise)
            p_succ = 1 - np.exp(-sinr / sinr_th)
            success[i] = np.random.rand() < p_succ
            sinr_values.append(sinr)
            interf -= p

    return success, sinr_values

In [ ]:
# ==========================================================
# REINFORCEMENT LEARNING
# ==========================================================
def rl_update(nodes, success):
    for i,n in enumerate(nodes):
        reward = (1 if success.get(i,False) else -1) - 0.05*n["power"]
        action = np.argmax(Q[i])

        Q[i,action] += alpha_rl * (
            reward + gamma_rl*np.max(Q[i]) - Q[i,action]
        )

        if action == 0: n["power"] *= 0.9
        elif action == 2: n["power"] *= 1.1

        n["power"] = np.clip(n["power"],0.01,Pmax)

In [49]:
# ==========================================================
# MAIN SIMULATION + SCORE CALCULATION
# ==========================================================
results_system = []
results_paper = []

for snr in SNR_dB:
    nodes = init_nodes()

    generated = delivered = 0
    total_energy = total_delay = 0
    sinr_log = []

    for _ in range(ROUNDS):
        channel_model(nodes)
        hmbo(nodes)
        build_tree(nodes)
        relay_selection(nodes)

        active = []
        for i in range(N):
            if np.random.rand() < packet_arrival_prob:
                nodes[i]["traffic"] += 1
                active.append(i)
                generated += 1

        if len(active) < 2:
            continue

        clusters = noma_clusters(active, nodes)
        success, sinrs = sic(nodes, clusters, snr)
        rl_update(nodes, success)
        sinr_log.extend(sinrs)

        for i in active:
            total_energy += nodes[i]["power"] * slot_time
            if success.get(i,False):
                delivered += 1
                total_delay += slot_time

    # ---------------- SYSTEM-LEVEL METRICS ----------------
    PDR = delivered / max(generated,1)
    throughput = (delivered * packet_size) / (ROUNDS * slot_time)
    EE_sys = (delivered * packet_size) / max(total_energy,1e-12)
    SE_eff = throughput / bandwidth
    latency = (total_delay / max(delivered,1)) * 1000

    results_system.append([snr, PDR, EE_sys, SE_eff, latency])

    # ---------------- PAPER-STYLE SCORES ----------------
    sinr_avg = np.mean(sinr_log)
    SE_paper = np.log2(1 + sinr_avg)
    EE_paper = SE_paper / (np.mean([n["power"] for n in nodes]))

    results_paper.append([snr, SE_paper, EE_paper])

results_system = np.array(results_system)
results_paper = np.array(results_paper)

In [50]:
# ==========================================================
# PRINT RESULTS
# ==========================================================
print("\nSYSTEM-LEVEL METRICS (REAL NETWORK)\n")
print("SNR  PDR   EE(bits/J)      SE_eff   Lat(ms)")
for r in results_system:
    print(f"{int(r[0]):<4} {r[1]:.3f} {r[2]:.2e}   {r[3]:.3f}   {r[4]:.2f}")



SYSTEM-LEVEL METRICS (REAL NETWORK)

SNR  PDR   EE(bits/J)      SE_eff   Lat(ms)
0    0.938 4.91e+06   0.152   10.00
5    0.936 5.02e+06   0.151   10.00
10   0.942 5.06e+06   0.160   10.00
15   0.933 5.05e+06   0.155   10.00
20   0.938 5.03e+06   0.153   10.00
25   0.937 4.95e+06   0.152   10.00
30   0.933 4.85e+06   0.153   10.00


In [51]:
import numpy as np

# ==========================================================
# FINAL RESULTS ARRAY (example – replace with your own)
# ==========================================================
results = np.array([

    [0,  0.938, 4.91e6, 0.152, 10.0],
    [5,  0.936, 5.02e6, 0.151, 10.0],
    [10, 0.942, 5.06e6, 0.160, 10.0],
    [15, 0.933, 5.05e6, 0.155, 10.0],
    [20, 0.938, 5.03e6, 0.153, 10.0],
    [25, 0.937, 4.95e6, 0.152, 10.0],
    [30, 0.933, 4.85e6, 0.153, 10.0]
])


# ==========================================================
# PRINT FINAL TABLE (PAPER FORMAT)
# ==========================================================
print("\nFINAL HMBO-TTPA-NOMA RESULTS\n")

print("{:<10} {:<8} {:<30} {:<34} {:<12}".format(
    "SNR (dB)",
    "PDR",
    "Energy Efficiency (bits/J)",
    "Spectral Efficiency (bits/s/Hz)",
    "Latency (ms)"
))

for row in results:
    print("{:<10.0f} {:<8.3f} {:<30.2f} {:<34.2f} {:<12.2f}".format(
        row[0], row[1], row[2], row[3], row[4]
    ))

# ==========================================================
# PRINT SINGLE-SNR DETAILED RESULT (e.g., 5 dB)
# ==========================================================
snr_target = 5

row = results[results[:,0] == snr_target][0]

print("\n")
print(f"SNR_dB:                              {int(row[0])} dB")
print(f"Packet Delivery Ratio (PDR):         {row[1]:.6f}  (~{row[1]*100:.0f}%)")
print(f"Energy Efficiency (bits/J):          {row[2]:.4f}")
print(f"Spectral Efficiency (bits/s/Hz):     {row[3]:.6f}")
print(f"Latency (ms):                        {row[4]:.6f}")



FINAL HMBO-TTPA-NOMA RESULTS

SNR (dB)   PDR      Energy Efficiency (bits/J)     Spectral Efficiency (bits/s/Hz)    Latency (ms)
0          0.938    4910000.00                     0.15                               10.00       
5          0.936    5020000.00                     0.15                               10.00       
10         0.942    5060000.00                     0.16                               10.00       
15         0.933    5050000.00                     0.15                               10.00       
20         0.938    5030000.00                     0.15                               10.00       
25         0.937    4950000.00                     0.15                               10.00       
30         0.933    4850000.00                     0.15                               10.00       


SNR_dB:                              5 dB
Packet Delivery Ratio (PDR):         0.936000  (~94%)
Energy Efficiency (bits/J):          5020000.0000
Spectral Efficiency (bits/s/H

In [52]:

print("SNR  SE(bits/s/Hz)  EE(bits/J)")
for r in results_paper:
    print(f"{int(r[0]):<4} {r[1]:.3f}          {r[2]:.2f}")

SNR  SE(bits/s/Hz)  EE(bits/J)
0    7.884          518.54
5    8.245          482.78
10   9.123          463.31
15   11.261          553.12
20   11.312          650.33
25   12.785          768.42
30   14.379          803.58
